In [3]:
# Use data.npz

import onnxruntime
import numpy as np

model_file = 'model.fixed2'

ort_session = onnxruntime.InferenceSession(model_file + '.onnx', providers=['CPUExecutionProvider'])
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
print(input_name, output_name)
print("Available providers:", ort_session.get_providers())
print("Current provider:", ort_session.get_provider_options())

# 加载npz文件中的数据
data = np.load(r'data\imagenet\validation\_data.npz')
images = data['images']
labels = data['labels']

# 获取模型输入的名称
input_name = ort_session.get_inputs()[0].name

# 对图像进行推理
outputs = ort_session.run(None, {input_name: images})

# 获取预测标签
predicted_labels = np.argmax(outputs[0], axis=1)

# 计算准确率
accuracy = (predicted_labels == labels).mean()

print(f'Accuracy: {accuracy * 100:.2f}%')


input output
Available providers: ['CPUExecutionProvider']
Current provider: {'CPUExecutionProvider': {}}
Accuracy: 77.33%


In [3]:
# Use src Image + _label.txt
import onnxruntime
import numpy as np
import torchvision.transforms as transforms
import os
from PIL import Image

model_file = 'resnet50'

ort_session = onnxruntime.InferenceSession(model_file + '.onnx', providers=['CPUExecutionProvider'])
input_name = ort_session.get_inputs()[0].name
output_name = ort_session.get_outputs()[0].name
print(input_name, output_name)
print("Available providers:", ort_session.get_providers())
print("Current provider:", ort_session.get_provider_options())

# 定义图像预处理步骤
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def preprocess_image(image_path):
    image = Image.open(image_path)
    image = image.convert('RGB')
    image = preprocess(image)
    image = image.unsqueeze(0)  # 添加批次维度
    return image

# 加载 labels.txt 文件并解析标签
def load_labels(labels_file):
    labels = {}
    with open(labels_file, 'r') as f:
        for line in f:
            image_name, label = line.strip().split(',')
            labels[image_name] = int(label)
    return labels

# 计算准确率
def calculate_accuracy(validation_folder, label_file):
    labels = load_labels(label_file)
    correct_predictions = 0
    total_images = 0

    for image_name in os.listdir(validation_folder):
        if image_name.endswith('.jpg'):
            image_path = os.path.join(validation_folder, image_name)
            image_data = preprocess_image(image_path)
            ort_inputs = {ort_session.get_inputs()[0].name: image_data.numpy()}
            ort_outs = ort_session.run(None, ort_inputs)
            predicted_label = np.argmax(ort_outs[0])

            if predicted_label == labels[image_name]:
                correct_predictions += 1
            total_images += 1

    accuracy = correct_predictions / total_images
    return accuracy

validation_dir = r'data\imagenet\validation'
calculate_accuracy(validation_dir, os.path.join(validation_dir, '_labels.txt'))

input output
Available providers: ['CPUExecutionProvider']
Current provider: {'CPUExecutionProvider': {}}


0.7666666666666667